<!--- Fancy Logo --->
<p align="center">
  <img src="https://em-content.zobj.net/source/microsoft-teams/363/robot_1f916.png" alt="Agentic AI Logo" width="100"/>
</p>

# 🤖 Agentic AI: Multi-Agent Sales Email Automation with Guardrails

 
In this notebook, you'll build a robust, multi-agent AI workflow for automating sales email generation and delivery. You'll explore how to orchestrate multiple LLM-powered agents, enforce input guardrails to prevent mistakes (like leaking personal names), and use structured outputs for reliable downstream processing.  
By the end, you'll have a system that can generate, select, and send professional sales emails—safely and efficiently—using the latest agentic AI techniques.


In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [6]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [7]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [ ]:
# Define a description string that will be used as the tool description for all three sales agent tools.
description = "Write a cold sales email"

# Convert the first sales agent (sales_agent1) into a tool using its as_tool method.
# The tool is named "sales_agent1" and uses the description defined above.
# This allows the agent to be invoked as a callable tool in an agentic workflow.
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)

# Similarly, convert the second sales agent (sales_agent2) into a tool.
# The tool is named "sales_agent2" and shares the same description.
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)

# Likewise, convert the third sales agent (sales_agent3) into a tool.
# The tool is named "sales_agent3" and also uses the same description.
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [ ]:
# The @function_tool decorator is used to turn a regular Python function into a tool that can be called by an agent.
# When you decorate a function with @function_tool, it registers the function as a tool with the agentic framework,
# allowing agents to invoke it as part of their workflow (e.g., to send an email, call an API, etc.).
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("adnanmdashikml@gmail.com")  # Change to your verified sender
    to_email = To("adnanmdashikml@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    # handoff_description is an optional string argument you can provide when defining an Agent that is intended to be a "handoff" target for another agent.
    # It describes, in natural language, what this agent does when another agent "hands off" a task to it.
    # This description helps the orchestrator (and the LLM) understand when and how to delegate a task to this agent.
    # For example, here: handoff_description="Convert an email to HTML and send it"
    # means: "If you have an email that needs to be converted to HTML and sent, hand it off to this agent."
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    # The 'handoffs' argument specifies which agents this agent can delegate tasks to.
    # Here, 'handoffs' is set to a list containing the 'emailer_agent', so the Sales Manager
    # can hand off the chosen email to the Email Manager agent for formatting and sending.
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [16]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

The `@input_guardrail` decorator in the **OpenAI Agent SDK** plays a critical role in ensuring that the agent's **inputs are safe, clean, or policy-compliant before execution**. Here's a breakdown of **why we need it** and **how it works**:

---

### ✅ Why We Need `@input_guardrail`

In the context of agentic workflows, agents interact with user inputs or data that may be:

* Unsafe (e.g., containing personal info like names or emails)
* Against policy (e.g., prompts triggering offensive language)
* Incorrectly formatted (e.g., malformed JSON, missing keys)
* Unwanted for a specific agent behavior (e.g., commands instead of questions)

The guardrail acts like a **pre-check filter or gatekeeper**, preventing an agent from executing if the input doesn't meet certain requirements or safety standards.

---

### ⚙️ How It Works

Let's step through what happens in your example:

```python
@input_guardrail  # <--- Decorator registers the function as an input validation step
async def guardrail_against_name(ctx, agent, message):
```

* **Registration**: The `@input_guardrail` decorator tells the Agent SDK that this function is an *input guardrail*. It will be **called before** the actual agent is invoked.

```python
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
```

* **Execution**: It runs another *guardrail agent* (`guardrail_agent`) to analyze the input `message`. This could be a specialized model that looks for things like:

  * Named Entities (e.g., personal names)
  * Sensitive topics
  * Profanity or abuse

```python
    is_name_in_message = result.final_output.is_name_in_message
```

* **Interpretation**: From the result, it checks if a personal name was found in the message.

```python
    return GuardrailFunctionOutput(
        output_info={"found_name": result.final_output},
        tripwire_triggered=is_name_in_message
    )
```

* **Tripwire**: If the name was found, `tripwire_triggered=True`. This tells the Agent SDK:

  > “⚠️ Don't let the main agent proceed — input violates the guardrail.”

---

### 🧠 Conceptually

Think of it like **middleware or a validator** in a web API:

| Guardrail Step            | Equivalent                                 |
| ------------------------- | ------------------------------------------ |
| `@input_guardrail`        | Decorator that defines a validator         |
| `Runner.run(...)`         | Executes a sub-agent or classifier         |
| `tripwire_triggered=True` | Return 403 / prevent agent from continuing |
| `output_info`             | Metadata or logs about what happened       |

---

### 🧪 Real Use Cases

You might use input guardrails for:

* Blocking questions with PII (e.g., “What’s John's address?”)
* Ensuring only specific formats (e.g., dates, JSON, SQL queries)
* Preventing re-entrant or recursive calls
* Custom moderation checks

In [18]:
@input_guardrail  # Decorator that marks this function as an input guardrail for agents
async def guardrail_against_name(ctx, agent, message):
    # Run the guardrail agent to check if a personal name is in the message
    # ctx.context provides additional context, agent is the target agent, message is user input
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    
    # Extract the boolean flag indicating if a name was found in the message
    # This comes from the NameCheckOutput model defined earlier
    is_name_in_message = result.final_output.is_name_in_message
    
    # Return a guardrail output with:
    # - output_info: stores the full result including the detected name
    # - tripwire_triggered: boolean that determines if the guardrail should block execution
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [19]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Check out the trace:

https://platform.openai.com/traces

In [20]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)